# This tutorial brings together all that we learned today

Using variable `tas`, masked over land, for `historical` over the 1980-2010 period.
Compute the model ensemble of the latest available version.
Compute departures from a 1970-2000 climatolgy for the each model ensemble average.

Show the spatial difference maps of these departures' standard deviation vs your choice of temperature observations. 

Do this for all models and plot them all on one plot, with the upper/left plot showing the obs standard_deviation.

**Bonus:** Produce the median model

# Step 1: Locate Models files

In [1]:
import genutil
import glob

template = '%(root)/%(collection)/%(type)/%(institution)/%(model)/%(experiment)/%(member)/%(table)/%(variable)/%(grid)/%(version)/%(variable)_%(table)_%(model)_%(experiment)_%(member)_%(grid)_%(period).nc'
path = genutil.StringConstructor(template)
for key in path.keys():
    setattr(path, key, "*")

path.root = '/global/cscratch1/sd/cmip6'  # this allows to easily change this from one machine to another
path.collection = 'CMIP6'
path.type = 'CMIP'
path.experiment = "historical"
path.table = "Amon"
path.variable = "tas"
print("path:", path())
all_files = glob.glob(path())
models = set()

for fname in all_files:
    sp = fname.split("/")
    models.add(sp[8])
print("models:", models)

path: /global/cscratch1/sd/cmip6/CMIP6/CMIP/*/*/historical/*/Amon/tas/*/*/tas_Amon_*_historical_*_*_*.nc
models: {'CNRM-CM6-1', 'CNRM-ESM2-1', 'HadGEM3-GC31-LL', 'GISS-E2-1-H', 'BCC-ESM1', 'EC-Earth3', 'GFDL-CM4', 'SAM0-UNICON', 'MIROC6', 'IPSL-CM6A-LR', 'CanESM5', 'GISS-E2-1-G', 'BCC-CSM2-MR', 'UKESM1-0-LL', 'CAMS-CSM1-0', 'CESM2-WACCM', 'EC-Earth3-Veg', 'CESM2', 'MRI-ESM2-0', 'GFDL-ESM4'}


# Step 2 produce departures from model ensemble

In [2]:
from subprocess import PIPE, Popen
import cdutil
import cdms2
import os

# turn off warnings
cdms2.setCompressionWarnings(0)

# Set parameters
start = '1980'
end = '2010'

climo_start = '1990'
climo_end = '2000'

# Define a template for our outputs
xml_name_template = "%(variable)_%(model)_%(member).%(extension)"
xml_name = genutil.StringConstructor(xml_name_template)

# Fix what we can on our template (variable at this point)
xml_name.variable = path.variable

# Loop through available models
for model in models:
    print("Processing:", model)
    # Fix some more param on our output name template
    xml_name.model = model
    xml_name.member = "avg"
    xml_name.extension = "nc"
    
    # Just for speed if we already generated the model ensemble average departures
    # Then skip it.
    if os.path.exists(xml_name()):
        print("Already here, skipping")
        continue
    
    # Using same template but with xml extension for cdscan files
    xml_name.extension = "xml"
    
    # Going to introspec versions and members available
    # For this model
    path.version = "*"  # all possible versions
    path.member = "*"  # All possible members
    path.model = model  # This model only
    # List available nc files
    model_files = glob.glob(path())
    # Prepare set/list of available versions (empty)
    versions = set()
    # Prepare set/list of available members (empty)
    members = set()
    for name in model_files:
        # split file name over slashes
        sp = name.split("/")
        # Figure out version and add to set
        versions.add(sp[-2])
        # Figure out member and add to set
        members.add(sp[-6])
    print("\tversions:", sorted(versions), sorted(members))
    # We will be using only the latest version
    path.version = sorted(versions)[-1]  # latest version
    
    # Initialize our output variable
    departures = None
    # Counter for number of members available
    nUsed = 0
    # Loop through members
    for member in members:
        # Fix member
        path.member = member
        # figure out all files that match this model/version/variable/member
        files = glob.glob(path())
        # Check thism member actually exists for this version
        # (at least one file)
        if len(files)>0:  # this member has data
            print("\tProcessing:", member, len(files))
            nUsed += 1 # increment counter for number of members
            # Fix member for output name
            xml_name.member = member
            if len(files)>1:
                # Ok if we have more than 1 netcdf file
                # We need to generate an xml file to link them
                
                # Let's construct cdscan command line
                cmd = "cdscan -x {} {}".format(xml_name(), " ".join(files))
                #print("\tCDSCAN COMMNAD:", cmd)
                # Run the cdscan command
                p = Popen(cmd.split(), stderr=PIPE, stdout=PIPE)
                o, e = p.communicate()
                # Open the just generated xml file
                f = cdms2.open(xml_name())
            else:
                # Only one file we can open it directly
                f = cdms2.open(files[0])
            # Read in period of interest
            data = f(path.variable, time=(start, end, "con"))
            f.close() # Close the file
            if len(files)>1:
                # Clean up unneeded xml file
                os.remove(xml_name())
            # compute climatology over climo period
            climo = cdutil.ANNUALCYCLE.climatology(data(time=(climo_start, climo_end, "con")))
            if departures is None:
                # First member
                # Departures from ref climo
                departures = cdutil.ANNUALCYCLE.departures(data, ref=climo)
            else:
                # Additional member we'll just add it to existing result
                # Departures from ref climo
                departures += cdutil.ANNUALCYCLE.departures(data, ref=climo)
    # We're done let's divide by the number of members used
    departures /= nUsed
    # Rest the extension to nc
    xml_name.extension = "nc"
    # Let's give this "member" value "avg"
    xml_name.member = "avg"
    # geenrate output name
    out = xml_name()
    print("\t\tWriting {} to {}".format(out, departures.shape))
    # Open for writing and dump
    fo = cdms2.open(out,'w')
    fo.write(departures, id=path.variable)
    fo.close()

Processing: CNRM-CM6-1
	versions: ['v20180917', 'v20181126', 'v20190125'] ['r10i1p1f2', 'r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 'r4i1p1f2', 'r5i1p1f2', 'r6i1p1f2', 'r7i1p1f2', 'r8i1p1f2', 'r9i1p1f2']
	Processing: r5i1p1f2 1
	Processing: r6i1p1f2 1
	Processing: r9i1p1f2 1
	Processing: r10i1p1f2 1
	Processing: r8i1p1f2 1
	Processing: r3i1p1f2 1
	Processing: r4i1p1f2 1
	Processing: r7i1p1f2 1
		Writing tas_CNRM-CM6-1_avg.nc to (360, 128, 256)
Processing: CNRM-ESM2-1
	versions: ['v20181206', 'v20190125'] ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 'r4i1p1f2', 'r5i1p1f2']
	Processing: r5i1p1f2 1
	Processing: r3i1p1f2 1
	Processing: r4i1p1f2 1
	Processing: r2i1p1f2 1
		Writing tas_CNRM-ESM2-1_avg.nc to (360, 128, 256)
Processing: HadGEM3-GC31-LL
Already here, skipping
Processing: GISS-E2-1-H
Already here, skipping
Processing: BCC-ESM1
Already here, skipping
Processing: EC-Earth3
Already here, skipping
Processing: GFDL-CM4
Already here, skipping
Processing: SAM0-UNICON
Already here, skipping
Processing:

# Step 3 departures for obs

Now let's do the same thing for the obs

In [3]:
# Obs file
obs_path = '/global/cscratch1/sd/cmip6/MERRA2/CREATE-IP.reanalysis.NASA-GMAO.MERRA-2.atmos.mon/tas_Amon_reanalysis_MERRA2_198001-201902.nc'
f = cdms2.open(obs_path)
# Read period of interest
tas = f("tas", time=(start, end, "con"))
f.close()
# Compute climo
climo = cdutil.ANNUALCYCLE.climatology(tas(time=(climo_start, climo_end, "con")))
# Departures from ref climo
departures = cdutil.ANNUALCYCLE.departures(tas, ref=climo)
# Dump to file
f = cdms2.open("tas_reanalysis_MERRA2.nc", "w")
f.write(departures, id="tas")
f.close()

# Step 4: Plotting

Let's count the models we used

In [4]:
n = len(models)
print("N Models:", n)

N Models: 20


With obs it's 21 models, let's plot this in portrait mode with 3 columns.

For this we will use EzTemplate

In [5]:
import EzTemplate
import vcs

# Prepare the canvas
canvas = vcs.init()

# Prepare a template
template = vcs.createtemplate()
# blank everything
template.blank()
# turn back on data/legend and dataname only
template.legend.priority = 1
template.data.priority = 1
template.dataname.priority=1

# Create the multi object
Multi = EzTemplate.Multi(rows=7, columns=3, x=canvas, template=template)

Let's compute the obs standard deviation

In [6]:
import genutil
std_obs = genutil.statistics.std(departures, axis="t")

For speed purpose let's put the obs on a T42 Grid

In [7]:
target_grid = cdms2.createGaussianGrid(64)
std_obs = std_obs.regrid(target_grid)(latitude=(-90,90))  # Flip it back

/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1313: Warning: 
avariable.regrid: We chose regridTool = esmf for you among the following choices:
   Tools ->    'regrid2' (old behavior)
               'esmf' (conserve, patch, linear) or
               'libcf' (linear)
  warnings.warn(message, Warning)
/opt/conda/lib/python3.7/site-packages/cdms2/avariable.py:1320: Warning: 
avariable.regrid: We chose regridMethod = linear for you among the following choices:
    'conserve' or 'linear' or 'patch'
  warnings.warn(message, Warning)


Let's generate the land/sea mask on that target grid.
And apply it to the obs

In [8]:
import MV2
msk = MV2.greater(cdutil.generateLandSeaMask(std_obs), .5)
std_obs = MV2.masked_where(msk, std_obs)
std_obs.id = "MERRA2"

Now let's get the first template, it usually has the legend on, let's turn it off
And let's use it to plot the obs

In [9]:
tmpl = Multi.get(row=0, column=0)
tmpl.legend.priority=0  # turn off legend
canvas.plot(std_obs, tmpl, ratio="autot")

Let's prepare some things to plot models

In [10]:
column = 1
row = 0
# Fix template parts that we can
xml_name.member = "avg"
xml_name.extension = "nc"
# Get the obs grid (T42 but flipped back to normal)
obs_grid = std_obs.getGrid()

# Create isofill graphics method
# And set levels and colors and extension arrows
isof = vcs.createisofill()
levels = vcs.mkscale(-1.,1.)
isof.levels = levels
isof.ext_1 = True
isof.ext_2 = True
isof.fillareacolors = vcs.getcolors(isof.levels)

Now let's loop through the models and plot them.

In [ ]:
for model in models:
    print("Processing:", model)
    # Fix the last moving part on the template
    xml_name.model = model
    f = cdms2.open(xml_name())
    # read in departures
    tas = f("tas")
    print("\t\t\t", tas.shape)
    # Conmpute std over time
    std = genutil.statistics.std(tas, axis='t')
    # Diff from obs
    # No need to apply mask since obs are masked
    diff = std.regrid(obs_grid) - std_obs
    # Set id for plot
    diff.id = model
    # Print diff min/max
    print("\t\t\t", genutil.minmax(diff))
    tmpl = Multi.get(row=row, column=column)
    if column==1 and row==0:
        # First model let's plot the legend
        tmpl.legend.priority=1  # turn on legend for this guy
    d = canvas.plot(diff, tmpl, isof, ratio="autot")
    column += 1
    if column == 3:
        column=0
        row+=1
d

Processing: CNRM-CM6-1
			 (360, 128, 256)
			 (-3.0847526543321484, 0.502313726123706)
Processing: CNRM-ESM2-1
			 (360, 128, 256)
			 (-2.8705920164531724, 0.978606157031076)
Processing: HadGEM3-GC31-LL
			 (360, 144, 192)
			 (-1.0568613284127966, 2.4589885168367203)
Processing: GISS-E2-1-H
			 (360, 90, 144)
			 (-2.95669228656891, 1.6185509068005466)
Processing: BCC-ESM1
			 (360, 64, 128)
			 (-2.1473977413026173, 2.1500887078809265)
Processing: EC-Earth3
			 (360, 256, 512)
			 (-0.9325096301361011, 3.6010233940339607)
Processing: GFDL-CM4
			 (360, 180, 288)
			 (-1.1838214385706194, 2.8154608710773124)
Processing: SAM0-UNICON
			 (360, 192, 288)
			 (-1.0340257543316747, 3.0913743821737194)
Processing: MIROC6
			 (360, 128, 256)
			 (-3.2085569185669, 0.15696635455780616)
Processing: IPSL-CM6A-LR
			 (360, 143, 144)
			 (-3.4316525278067957, -0.08366607075016774)
Processing: CanESM5
			 (360, 64, 128)
			 (-3.552567039612684, 0.9202409345398943)
Processing: GISS-E2-1-G
			 (30

In [12]:
canvas.png("final")

![](final.png)